In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,recollection,class,forty,beauty,incessantly,vigour,astonished,double,waiting,paint,...,dodo,correspondent,northumbria,fine,excessively,middle,curtain,cheer,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9877742946708464

Test set score: 0.8881578947368421


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.9579937304075236

Test set score: 0.9158834586466166


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.886833855799373

Test set score: 0.8735902255639098


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [14]:
emma_bow.shape[0]

1669

In [15]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6976137211036539


col_0,Austen,Carroll
row_0,,
Austen,1564,105
Carroll,706,307


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

# Challenge 0: #

In [16]:
from sklearn.svm import SVC
svc = SVC()
trainsvc = svc.fit(X_train, y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.6824451410658308

Test set score: 0.6917293233082706


In [34]:
svc = SVC(C=.35,kernel='linear')
trainsvc = svc.fit(X_train, y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.95141065830721

Test set score: 0.9088345864661654


In [19]:
#  SVC doesn't work very well
# Let's try lasso regression

#from sklearn import linear_model

lams = [0.01,.1,.2,.3,.4,.5,.6,.7,.8,.9,1.0]

for lam in lams:
    ridge = LogisticRegression(C=lam, penalty='l2')
    ridge.fit(X_train, y_train)
    
    lasso = LogisticRegression(C=lam, penalty='l1')
    lasso.fit(X_train, y_train)
    
    print("Ridge scoring for {:3f} is {:3f}".format(lam, ridge.score(X_test, y_test)))
    print("Lasso scoring for {:3f} is {:3f}".format(lam, lasso.score(X_test, y_test)))


Ridge scoring for 0.010000 is 0.815320
Lasso scoring for 0.010000 is 0.761278
Ridge scoring for 0.100000 is 0.882989
Lasso scoring for 0.100000 is 0.838816
Ridge scoring for 0.200000 is 0.896147
Lasso scoring for 0.200000 is 0.867011
Ridge scoring for 0.300000 is 0.903665
Lasso scoring for 0.300000 is 0.880169
Ridge scoring for 0.400000 is 0.908365
Lasso scoring for 0.400000 is 0.889568
Ridge scoring for 0.500000 is 0.912594
Lasso scoring for 0.500000 is 0.892387
Ridge scoring for 0.600000 is 0.913534
Lasso scoring for 0.600000 is 0.897556
Ridge scoring for 0.700000 is 0.915414
Lasso scoring for 0.700000 is 0.900376
Ridge scoring for 0.800000 is 0.916823
Lasso scoring for 0.800000 is 0.902256
Ridge scoring for 0.900000 is 0.915883
Lasso scoring for 0.900000 is 0.903665
Ridge scoring for 1.000000 is 0.915883
Lasso scoring for 1.000000 is 0.905075


In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

# scale the input into mean 0 and standardized variance
#scaled_X = StandardScaler().fit_transform(X_train)

pca = PCA(n_components=20)
newX = pca.fit_transform(X_train)

print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.06881961 0.0219588  0.01541367 0.01282655 0.01120434 0.00983003
 0.00939604 0.00891423 0.0086269  0.00826917 0.00807549 0.00778993
 0.00753906 0.0070584  0.006909   0.00644973 0.00640221 0.00602319
 0.00577078 0.00569941]
[45.06828565 25.45770517 21.32887813 19.45672986 18.18477752 17.03304757
 16.65280475 16.22022358 15.95667279 15.62233601 15.43830025 15.16287981
 14.91672231 14.43337676 14.27980911 13.797028   13.74611117 13.3330077
 13.05064778 12.96969652]


In [25]:
            
X_pcatest = pca.transform(X_test)

In [26]:
lams = [0.01,.1,.2,.3,.4,.5,.6,.7,.8,.9,1.0]

for lam in lams:
    ridge = LogisticRegression(C=lam, penalty='l2')
    ridge.fit(newX, y_train)
    
    lasso = LogisticRegression(C=lam, penalty='l1')
    lasso.fit(newX, y_train)
    
    print("Ridge scoring for {:3f} is {:3f}".format(lam, ridge.score(X_pcatest, y_test)))
    print("Lasso scoring for {:3f} is {:3f}".format(lam, lasso.score(X_pcatest, y_test)))


Ridge scoring for 0.010000 is 0.811090
Lasso scoring for 0.010000 is 0.796992
Ridge scoring for 0.100000 is 0.846335
Lasso scoring for 0.100000 is 0.854323
Ridge scoring for 0.200000 is 0.850564
Lasso scoring for 0.200000 is 0.862312
Ridge scoring for 0.300000 is 0.856203
Lasso scoring for 0.300000 is 0.864662
Ridge scoring for 0.400000 is 0.855733
Lasso scoring for 0.400000 is 0.869831
Ridge scoring for 0.500000 is 0.858553
Lasso scoring for 0.500000 is 0.870771
Ridge scoring for 0.600000 is 0.860432
Lasso scoring for 0.600000 is 0.870771
Ridge scoring for 0.700000 is 0.861842
Lasso scoring for 0.700000 is 0.873120
Ridge scoring for 0.800000 is 0.863722
Lasso scoring for 0.800000 is 0.873590
Ridge scoring for 0.900000 is 0.864662
Lasso scoring for 0.900000 is 0.873590
Ridge scoring for 1.000000 is 0.865132
Lasso scoring for 1.000000 is 0.874530


In [55]:
rfc = ensemble.RandomForestClassifier(max_features=15,n_estimators=500)
from sklearn.model_selection import cross_val_score
#cross_val_score(rfc, X_train, y_train, cv=10)

In [56]:
train = rfc.fit(X_train, y_train)
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9937304075235109

Test set score: 0.9219924812030075


In [58]:
# Properly tuned Random Forest can beat Logistic Regression
# But it's Hard!!
# The key appears to be max_features to a small number
# and number of trees greater than 100

# Challenge 1: #

In [61]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [ ]:
# Let's compare Lewis Carroll with Jane Austen's Sense & Sensibility

In [62]:
# Clean the sense&Sensibility data.
sense = gutenberg.raw('austen-sense.txt')
sense = re.sub(r'VOLUME \w+', '', sense)
sense = re.sub(r'CHAPTER \w+', '', sense)
sense = text_cleaner(sense)
print(sense[:100])

The family of Dashwood had long been settled in Sussex. Their estate was large, and their residence 


In [63]:
len(alice_sents)

1669

In [64]:
# Parse our cleaned data.
sense_doc = nlp(sense)

In [65]:
# Group into sentences.
#persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
sense_sents = [[sent, "Austen"] for sent in sense_doc.sents]

# Sense is quite long, let's cut it down to the same length as Alice.
sense_sents = sense_sents[0:len(alice_sents)]

In [66]:
# Build a new Bag of Words data frame for Sense word counts.
# We'll use the same common words from Alice and Persuasion.
sense_sentences = pd.DataFrame(sense_sents)
sense_bow = bow_features(sense_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [67]:
# Now we can model it!
# Let's use our Random Forest Classifier again.

# Combine the Sense sentence data with the Alice data from the test set.
X_Sense_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    sense_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Sense_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * sense_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Sense_test, y_Sense_test))
rfc_Sense_predicted = rfc.predict(X_Sense_test)
pd.crosstab(y_Sense_test, rfc_Sense_predicted)


Test set score: 0.6823266219239373


col_0,Austen,Carroll
row_0,,
Austen,1461,208
Carroll,669,344


In [68]:
# Similar to other Jane Austen novels.